# To-Do List

1. Explore different embeddings
2. Add better metadata to chunks

# Import Libraries

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_community.document_loaders import WebBaseLoader, LocalBaseLoader, LocalFolderLoader, LocalFileLoader, LocalFolderLoader
from langchain_community.vectorstores import Chroma
from groq import Groq
from langchain_groq import ChatGroq
from src.utils import *
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
import os
from dotenv import load_dotenv

utils.py loaded


In [2]:
# import api keys
# .\qrag-env\Scripts\activate # command to activate virtual environment in terminal
load_dotenv()
groq_api_key = os.getenv("groq_api_key")
openai_api_key = os.getenv("chatgpt.gpt3.apiKey")
mistra_api_key = os.getenv("mistra_api_key")
tavily_api_key = os.getenv("tavily_api_key")

# Instantiate Embedding Model

The choice of the embedding model is important and depends on the RAG application. Since a specific LLM is trained using a particular embedding model, the choice of LLM in the RAG application dictates the choice of the embedding model. Here is a table of LLM's and their embedding models. 

| Model                        | Benchmark/Metric          | Performance Highlights                                                                 |
|------------------------------|---------------------------|----------------------------------------------------------------------------------------|
| SPECTER2                     | SciRepEval                | Outperforms general-purpose models on regression, classification, and re-ranking tasks. |
| Snowflake Arctic Embed       | MTEB Retrieval Leaderboard| Ranks first among models of similar size; large model outperforms models with 1B+ parameters. |
| Tabular Embedding Model (TEM)| Domain-Specific Performance| Excels in handling high-dimensional datasets, particularly in financial and scientific domains. |
| Voyage AI Embedding Models   | RAG Implementation        | High retrieval quality and efficiency in generating accurate responses[4].              |
| OpenAI text-embedding-3-large| Embedding Dimensions      | High performance in multi-language retrieval and task-specific accuracy[11].            |


In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain_mistralai import MistralAIEmbeddings

openai = OpenAIEmbeddings(model='text-embedding-3-large',api_key=openai_api_key)
mistral = MistralAIEmbeddings(
    model="mistral-embed",
    api_key=mistra_api_key,
)
# embed_model = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

c:\Users\evang\Desktop\GraduateSchool\Graduation\qRAG\qrag-env\Lib\site-packages\langchain_mistralai\embeddings.py:105: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


# Instantiate LLM

In [4]:
llm = ChatGroq(temperature=0,
              # model_name="Llama3-70b-8192",
              model_name="mixtral-8x7b-32768",
              api_key=groq_api_key)

# Import Data (Theses, Papers)

Since the data consists of pdf documents, they first need to be parsed, split into chunks, and then embedded. There are multiple packages that can be used to load and parse pdf's, like pdflumber, pypdf, and pymupdf. It seems like pymypdf is the best according to the online literature. Also, pymupdf seems to have extensive documentation and support, unlike pdflumber, including examples of using pymupdf for RAG applications. Neat! 

If you want to learn more, [https://pymupdf.readthedocs.io/en/latest/index.html](URL)

In [5]:
theses_paths = get_file_paths(r"Theses/")
papers_paths = get_file_paths(r"Papers/")

Theses\intro-to-quantum-measurement.pdf
Theses\Mivev-Thesis.pdf
Theses\Schuster-Thesis.pdf
Theses\Schwarz-Thesis.pdf
Theses\Slichter-Thesis.pdf
Papers\oliver-QME.pdf
Papers\qe-guide-scqubits-oliver.pdf
Papers\wendin-review-scqubits.pdf


In [ ]:
from langchain_community.document_loaders import DirectoryLoader
theses = load_pdf_document(theses_paths)
papers = load_pdf_document(papers_paths)

## Save markdown

In [ ]:
theses[0]

In [ ]:
save_markdown_text(theses,theses_paths)
save_markdown_text(papers,papers_paths)

## Load Markdown Documents

In [6]:
md_theses = load_markdown_text(theses_paths)
md_papers = load_markdown_text(papers_paths)

Loading markdown text from: Theses\intro-to-quantum-measurement.md
Loading markdown text from: Theses\Mivev-Thesis.md
Loading markdown text from: Theses\Schuster-Thesis.md
Loading markdown text from: Theses\Schwarz-Thesis.md
Loading markdown text from: Theses\Slichter-Thesis.md
Loading markdown text from: Papers\oliver-QME.md
Loading markdown text from: Papers\qe-guide-scqubits-oliver.md
Loading markdown text from: Papers\wendin-review-scqubits.md


## Split into chunks

Documents are split into chunks depending on the headers. Useful metadata such as chunk type (chapter, section, subsection, etc.) and name are created. Further splitting according to meaning is done after that to enchance chunking and group chunks according to similarity in embedding space. This method is better compared to brute-force splitting where the only input parameters are the chunk_size and chunk_overlap. Keep in mind that the final chunk size needs to conform to the size of the embedding model. For more information, here are some useful links.

- **Semantic Splitting with Langchain**: https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/semantic-chunker/
- **Information on how to choose the right embedding model**: https://datasciencedojo.com/blog/choosing-the-right-vector-embedding-model/

In [7]:
# Splitting the documents into chunks based on the headers
headers_to_split_thesis_on = [
    ("#", "Chapter"),
    ("##", "Section"),
    ("###", "Subsection"),
    ("####", "Subsubsection"),
    ("#####", "Subsubsubsection"),
    # ("#"*6, "Subsubsubsubsection")
]

theses_chunks = split_docs(md_theses, headers_to_split_thesis_on)
papers_chunks = split_docs(md_papers, headers_to_split_thesis_on)
chunks = theses_chunks + papers_chunks

5 documents split into 227 chunks
3 documents split into 35 chunks


**Further Chunking Based on Meaning (Not functional yet)**

In [ ]:
# Splitting documents based on semantic similarity
from langchain_experimental.text_splitter import SemanticChunker

semantic_text_splitter = text_splitter = SemanticChunker(
    OpenAIEmbeddings(), breakpoint_threshold_type="interquartile",
)
docs = semantic_text_splitter.create_documents(papers_chunks)

# Vector Database

In [8]:
vectorstore = Chroma.from_documents(documents=chunks,
                                    embedding=openai,
                                    collection_name="local-rag")

# Setup Agents

In [35]:
# import time
# from langchain.prompts import PromptTemplate
# from langchain_core.output_parsers import JsonOutputParser
# from langchain_core.output_parsers import StrOutputParser



# prompt = PromptTemplate(
#     template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an expert at routing a 
#     user question to a vectorstore or web search. Use the vectorstore for questions on LLM  agents, 
#     prompt engineering, and adversarial attacks. You do not need to be stringent with the keywords 
#     in the question related to these topics. Otherwise, use web-search. Give a binary choice 'web_search' 
#     or 'vectorstore' based on the question. Return the a JSON with a single key 'datasource' and 
#     no premable or explaination. Question to route: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
#     input_variables=["question"],
# )
# # start = time.time()
# question_router = prompt | llm | JsonOutputParser()

# prompt = PromptTemplate(
#     template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks. 
#     Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. 
#      <|eot_id|><|start_header_id|>user<|end_header_id|>
#     Question: {question} 
#     Context: {context} 
#     Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
#     input_variables=["question", "document"],
# )

# # Post-processing
# def format_docs(docs):
#     return "\n\n".join(doc.page_content for doc in docs)

# # Chain
# # start = time.time()
# rag_chain = prompt | llm | StrOutputParser()

# prompt = PromptTemplate(
#     template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing relevance 
#     of a retrieved document to a user question. If the document contains keywords related to the user question, 
#     grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
#     Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
#     Provide the binary score as a JSON with a single key 'score' and no premable or explaination.
#      <|eot_id|><|start_header_id|>user<|end_header_id|>
#     Here is the retrieved document: \n\n {document} \n\n
#     Here is the user question: {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
#     """,
#     input_variables=["question", "document"],
# )
# # start = time.time()
# retrieval_grader = prompt | llm | JsonOutputParser()

# prompt = PromptTemplate(
#     template=""" <|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether 
#     an answer is grounded in / supported by a set of facts. Give a binary 'yes' or 'no' score to indicate 
#     whether the answer is grounded in / supported by a set of facts. Provide the binary score as a JSON with a 
#     single key 'score' and no preamble or explanation. <|eot_id|><|start_header_id|>user<|end_header_id|>
#     Here are the facts:
#     \n ------- \n
#     {documents} 
#     \n ------- \n
#     Here is the answer: {generation}  <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
#     input_variables=["generation", "documents"],
# )
# # start = time.time()
# hallucination_grader = prompt | llm | JsonOutputParser()

# prompt = PromptTemplate(
#     template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether an 
#     answer is useful to resolve a question. Give a binary score 'yes' or 'no' to indicate whether the answer is 
#     useful to resolve a question. Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
#      <|eot_id|><|start_header_id|>user<|end_header_id|> Here is the answer:
#     \n ------- \n
#     {generation} 
#     \n ------- \n
#     Here is the question: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
#     input_variables=["generation", "question"],
# )
# # start = time.time()
# answer_grader = prompt | llm | JsonOutputParser()
from agents import crew, agent_prompts
from langchain_community.tools.tavily_search import TavilySearchResults

agents = crew(agent_prompts,llm)
agents.build_crew()
web_search_tool = TavilySearchResults(k=3)

Building agent: question_router
Building agent: rag_chain
Building agent: retrieval_grader
Building agent: hallucination_grader
Building agent: answer_grader


# Create Graph

In [36]:
# from typing_extensions import TypedDict
# from typing import List

# ### State

# class GraphState(TypedDict):
#     question : str
#     generation : str
#     web_search : str
#     documents : List[str]

# from langchain.schema import Document
# def retrieve(state):
#     """
#     Retrieve documents from vectorstore

#     Args:
#         state (dict): The current graph state

#     Returns:
#         state (dict): New key added to state, documents, that contains retrieved documents
#     """
#     print("---RETRIEVE---")
#     question = state["question"]

#     # Retrieval
#     documents = retriever.invoke(question)
#     return {"documents": documents, "question": question}
# #
# def generate(state):
#     """
#     Generate answer using RAG on retrieved documents

#     Args:
#         state (dict): The current graph state

#     Returns:
#         state (dict): New key added to state, generation, that contains LLM generation
#     """
#     print("---GENERATE---")
#     question = state["question"]
#     documents = state["documents"]
    
#     # RAG generation
#     generation = rag_chain.invoke({"context": documents, "question": question})
#     return {"documents": documents, "question": question, "generation": generation}
# #
# def grade_documents(state):
#     """
#     Determines whether the retrieved documents are relevant to the question
#     If any document is not relevant, we will set a flag to run web search

#     Args:
#         state (dict): The current graph state

#     Returns:
#         state (dict): Filtered out irrelevant documents and updated web_search state
#     """

#     print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
#     question = state["question"]
#     documents = state["documents"]
    
#     # Score each doc
#     filtered_docs = []
#     web_search = "No"
#     for d in documents:
#         score = retrieval_grader.invoke({"question": question, "document": d.page_content})
#         grade = score['score']
#         # Document relevant
#         if grade.lower() == "yes":
#             print("---GRADE: DOCUMENT RELEVANT---")
#             filtered_docs.append(d)
#         # Document not relevant
#         else:
#             print("---GRADE: DOCUMENT NOT RELEVANT---")
#             # We do not include the document in filtered_docs
#             # We set a flag to indicate that we want to run web search
#             web_search = "Yes"
#             continue
#     return {"documents": filtered_docs, "question": question, "web_search": web_search}
# #
# def web_search(state):
#     """
#     Web search based based on the question

#     Args:
#         state (dict): The current graph state

#     Returns:
#         state (dict): Appended web results to documents
#     """

#     print("---WEB SEARCH---")
#     question = state["question"]
#     documents = state["documents"]

#     # Web search
#     docs = web_search_tool.invoke({"query": question})
#     web_results = "\n".join([d["content"] for d in docs])
#     web_results = Document(page_content=web_results)
#     if documents is not None:
#         documents.append(web_results)
#     else:
#         documents = [web_results]
#     return {"documents": documents, "question": question}

# def route_question(state):
#     """
#     Route question to web search or RAG.

#     Args:
#         state (dict): The current graph state

#     Returns:
#         str: Next node to call
#     """

#     print("---ROUTE QUESTION---")
#     question = state["question"]
#     print(question)
#     source = question_router.invoke({"question": question})  
#     print(source)
#     print(source['datasource'])
#     if source['datasource'] == 'web_search':
#         print("---ROUTE QUESTION TO WEB SEARCH---")
#         return "websearch"
#     elif source['datasource'] == 'vectorstore':
#         print("---ROUTE QUESTION TO RAG---")
#         return "vectorstore"
# def decide_to_generate(state):
#     """
#     Determines whether to generate an answer, or add web search

#     Args:
#         state (dict): The current graph state

#     Returns:
#         str: Binary decision for next node to call
#     """

#     print("---ASSESS GRADED DOCUMENTS---")
#     question = state["question"]
#     web_search = state["web_search"]
#     filtered_documents = state["documents"]

#     if web_search == "Yes":
#         # All documents have been filtered check_relevance
#         # We will re-generate a new query
#         print("---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---")
#         return "websearch"
#     else:
#         # We have relevant documents, so generate answer
#         print("---DECISION: GENERATE---")
#         return "generate"
    
# def grade_generation_v_documents_and_question(state):
#     """
#     Determines whether the generation is grounded in the document and answers question.

#     Args:
#         state (dict): The current graph state

#     Returns:
#         str: Decision for next node to call
#     """

#     print("---CHECK HALLUCINATIONS---")
#     question = state["question"]
#     documents = state["documents"]
#     generation = state["generation"]

#     score = hallucination_grader.invoke({"documents": documents, "generation": generation})
#     grade = score['score']

#     # Check hallucination
#     if grade == "yes":
#         print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
#         # Check question-answering
#         print("---GRADE GENERATION vs QUESTION---")
#         score = answer_grader.invoke({"question": question,"generation": generation})
#         print(generation)
#         grade = score['score']
#         if grade == "yes":
#             print("---DECISION: GENERATION ADDRESSES QUESTION---")
#             return "useful"
#         else:
#             print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
#             return "not useful"
#     else:
#         pprint("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
#         return "not supported"
        
# from langgraph.graph import END, StateGraph
# workflow = StateGraph(GraphState)

# # Define the nodes
# workflow.add_node("retrieve", retrieve) # retrieve
# workflow.add_node("websearch", web_search) # web search
# workflow.add_node("grade_documents", grade_documents) # grade documents
# workflow.add_node("generate", generate) # generatae

# workflow.set_conditional_entry_point(
#     route_question,
#     {
#         "websearch": "websearch",
#         "vectorstore": "retrieve",
#     },
# )

# workflow.add_edge("retrieve", "grade_documents")
# workflow.add_conditional_edges(
#     "grade_documents",
#     decide_to_generate,
#     {
#         "websearch": "websearch",
#         "generate": "generate",
#     },
# )
# workflow.add_edge("websearch", "generate")
# workflow.add_conditional_edges(
#     "generate",
#     grade_generation_v_documents_and_question,
#     {
#         "not supported": "generate",
#         "useful": END,
#         "not useful": "websearch",
#     },
# )
from graph import RAG_graph
from nodes import nodes

# actions = nodes(agents.crew, web_search_tool)
rag_graph = RAG_graph(agents.crew, web_search_tool, vectorstore)

# Compile & Test Workflow

In [34]:
rag_graph.actions.crew

{'router': PromptTemplate(input_variables=['question'], template="<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an expert at routing a \n            user question to a vectorstore or web search. Use the vectorstore for questions on LLM  agents, \n            prompt engineering, and adversarial attacks. You do not need to be stringent with the keywords \n            in the question related to these topics. Otherwise, use web-search. Give a binary choice 'web_search' \n            or 'vectorstore' based on the question. Return the a JSON with a single key 'datasource' and \n            no premable or explaination. Question to route: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>")
 | ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000250993F58B0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002509AA62570>, temperature=1e-08, groq_api_key=SecretStr('**********'))
 | JsonOutputParser(),
 

In [37]:
from langchain_core.runnables import RunnableConfig
app = rag_graph.compile()
config = RunnableConfig(recursion_limit=3)

In [38]:
from pprint import pprint
inputs = {"question": "What are the main requirements for being able to detect quantum jumps?"}
for output in app.stream(inputs,config):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
pprint(value["generation"])

---ROUTE QUESTION---
What are the main requirements for being able to detect quantum jumps?
{'datasource': 'web_search'}
web_search
---ROUTE QUESTION TO WEB SEARCH---
---WEB SEARCH---
'Finished running: websearch:'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
The main requirements for being able to detect quantum jumps are:

1. Performing a tomography measurement after a specific time interval (Δ tcatch).
2. Having the ability to catch the quantum jump, which involves detecting an advance warning signal. This is compared to predicting the eruption of a volcano, which is unpredictable in the long-term but may have detectable precursors.

These requirements are based on the information provided in the context, which discusses the observation and prediction of quantum jumps in various systems.
---DECISION: GENERATION ADDRESSES QUESTION---
'Finished running: generate:'
('The main requirements for being able